# Java 调用图生成工具的比较

这篇文章主要是为了比较，看不同调用图生成工具跑出来的调用图有哪些区别。

## 1. 引言部分

调用图 (`Call Graph`) 是一种描述过程间控制流与数据流的工具。图中节点表示方法，图中有向边连接两个节点，用于表示 **起始节点对应的方法** 调用了 **目标节点对应的方法**。

**静态图往往过分估计了调用图中边的数量**。这是因为在 OOP 语言中，一个方法的调用与 **运行时** `receiver` 的类型有关。所以，在创建静态图的时候，经常需要确定一个类型的对象可能会调用哪些方法。确定上述 **可能调用方法集合** 的算法是导致调用图产生区别的主要原因之一。

但还有许多类似影响调用图结构的要素。（文中举了个例子，比如匿名类的初始化方法选择（我不懂为什么啊））

接着，文中举出了 3 点：

1. 处理 `Java` 语言中语言特性的方式，如何影响调用图结构；
2. 实际调用图结构的差别体现在哪里；
3. 忽略某些差别，调用图结构是否相同（啥意思？）。

## 2. 相关工作

第二部分对调用图相关的文献进行了讨论。

调用图是刻画 `high level` 控制流的一种方法。不管被分析的语言是低级的二进制、汇编或是高级的 OO，调用图的构造总是一个问题。

首先是准确性，如果一个调用图把一个程序在动态执行时所有调用的方法都 “准确无误” 地指出了（包括多态也指定特定的方法），那就称之为 “准确”。然而，静态分析做不到这样准确。

对于调用对象方法的 `virtual call` 这个概念，就是无法做到 “准确” 的情形之一。针对这种问题，有两种方法，**上下文有关 / 无关分析方法的选择** 是 **影响调用图结构的因素之一**。

1. 上下文有关分析：比较准确，因为是按照顺序的。
2. 上下文无关分析：CHA / RTA / XTA / VTA 等算法被用于在 “准确性” 与 “正确性” 间做取舍。

另外一个因素是 **对库函数的处理**。

如果增加了库函数部分，那图会变得很大。但如果不加入的话，当在开发底层接口时，分析起来就没用（因为开发的就是那个玩意儿，环境就是它）。

决定 **处理库函数** 后，通常会把 `private-call / inaccessable-call` 也加入进去。通常使用的 `CHA / RTA` 算法没有考虑库函数。

文中说：

> . The tools we selected for our
comparison represent library calls and library methods at various levels of detail.

接着就是说了些其他人的研究成果。比如 C++ 的调用图构建方式；JavaScript 静态调用图创建工具；给出提升了准确性的 RTA 算法；比较了 Soot 和 \*J 构建的图，给出了比较调用图间区别的算法（Comparing call graphs. In ´ ACM SIGPLAN/SIGSOFT Workshop on Program Analysis for Software Tools and Engineering, pages 37–42）。

## 3. 调用图创建工具

第三部分比较了 6 种工具。

### 3.1 Soot

一种可以操作（比如修改）以及优化 IR 的工具。文中给的是 2019 的 Soot，支持的语言特性，然后我看了下截至目前应该也是只到支持 9。使用的是 CHA。

### 3.2 OpenStaticAnalyzer

不仅仅对 Java，还可以处理 JavaScript / Python / C#。处理了递归目录问题，在程序分析时考虑构建系统的影响。通过遍历 AST 获取 `invocation information` 来创建调用图。

### 3.3 SPOON

支持 Java 9。由于只是 Java 的，就根据已有目录分析，构建类似 AST 的模型（感觉在这里算是一种 IR）。本文说这个东西 **Well-documented**，吼！

### 3.4 Java Call Graph



## 4. 

第四部分给出了工具的评估标准。

## 5. 

第五部分对评估结果的量化指标进行了解释说明。

## 6. 

第六部分给出了可能影响结果的几个因素。但我想我暂时不会看这个。

## Note

| English | Chinese |
|:-:|:-:|
| imprecisions | 不确定性 |
| extensive literature | 大量文献 |
| mitigate | 减轻 |